In [1]:
%config IPCompleter.evaluation='unsafe'

# GenericJob class
Already discussed in the initial pyiron paper in Appendix C, the code is slightly update to be compatible with the recent version of `pyiron_base`. The two main parts are the `write_input()` function and the `collect_output()` function. 

In [2]:
import os
from pyiron_base import GenericJob, GenericParameters

In [3]:
class ToyJob(GenericJob):
    def __init__(self, project, job_name):
        super(ToyJob, self).__init__(project, job_name)
        self.input = ToyInput()
        self.executable = "cat input > output"

    def write_input(self):
        self.input.write_file(
            file_name="input",
            cwd=self.working_directory,
        )

    def collect_output(self):
        file = os.path.join(self.working_directory, "output")
        with open(file) as f:
            line = f.readlines()[0]
            energy = float(line.split()[1])
        with self.project_hdf5.open("output/generic") as h5out:
            h5out["energy_tot"] = energy

    def to_hdf(self, hdf=None, group_name=None):
        super(ToyJob, self).to_hdf(
            hdf=hdf,
            group_name=group_name
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.to_hdf(h5in)

    def from_hdf(self, hdf=None, group_name=None):
        super(ToyJob, self).from_hdf(
            hdf=hdf,
            group_name=group_name,
        )
        with self.project_hdf5.open("input") as h5in:
            self.input.from_hdf(h5in)

In [4]:
class ToyInput(GenericParameters):
    def __init__(self, input_file_name=None):
        super(ToyInput, self).__init__(
            input_file_name=input_file_name,
            table_name="input"
        )
    
    def load_default(self):
        self.load_string("input_energy 100")

Use the newly created job in a project 

In [5]:
from pyiron_base import Project

In [6]:
pr = Project('test')

In [7]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
job = pr.create_job(ToyJob, job_name="toy")

In [9]:
job.input

,Parameter,Value,Comment
0,input_energy,100,


In [10]:
job.run()

The job toy was saved and received the ID: 157


In [11]:
job.content['output/generic/energy_tot']

100.0

# Template Job

In [12]:
from pyiron_base import TemplateJob

In [13]:
class ToyTemplateJob(TemplateJob):
    def __init__(self, project, job_name):
        super(ToyTemplateJob, self).__init__(project, job_name)
        self.executable = "cat input > output"

    def write_input(self):
        with open(os.path.join(self.working_directory, "input"), "w") as f:
            f.write(str(job.input.energy))

    def collect_output(self):
        with open(os.path.join(self.working_directory, "output"), "r") as f:
            job.output.energy = f.read()
        job.to_hdf()

In [14]:
pr = Project('test')

In [15]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
job = pr.create_job(ToyTemplateJob, job_name="toytemp")

In [17]:
job.input.energy = 100

In [18]:
job.run()

The job toytemp was saved and received the ID: 157


In [19]:
job.output.energy

'100'

# Python Template Job
For jobs which primarily consist of an existing Python function.

In [20]:
from pyiron_base import PythonTemplateJob

In [21]:
def calculation(value):
    return 2 * value

In [22]:
class ToyJob(PythonTemplateJob):
    def __init__(self, project, job_name):
        super().__init__(project, job_name) 
        self.input.value = None

    def run_static(self):
        self.output.result = calculation(self.input.value)
        self.status.finished = True
        self.to_hdf()

In [23]:
pr = Project('template')

In [24]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/2 [00:00<?, ?it/s]

In [25]:
job = pr.create_job(ToyJob, job_name="toy")

In [26]:
job.input.value = 5

In [27]:
job.run()

The job toy was saved and received the ID: 158


In [28]:
job.output

DataContainer({'result': 10})

In [29]:
job.output.result

10

# For testing

In [30]:
def write_input(input_dict, working_directory):
    with open(os.path.join(working_directory, "input"), "w") as f:
        f.write(str(input_dict["energy"]))

In [31]:
def collect_output(working_directory):
    with open(os.path.join(working_directory, "output"), "r") as f:
        return {
            "energy": f.read()
        }

In [32]:
pr = Project('template')

In [33]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [34]:
pr.create_job_class(
    class_name="ToyJob",
    write_input_funct=write_input,
    collect_output_funct=collect_output,
    default_input_dict={  # Default Parameter 
        "energy": 100.0, 
    },
    executable_str="cat input > output",
)

In [35]:
job = pr.create.job.ToyJob(job_name="toy")

In [36]:
job.input

DataContainer({'energy': 100.0})

In [37]:
job.run()

The job toy was saved and received the ID: 158


In [38]:
job.output

DataContainer({'stdout': '', 'energy': '100.0'})

In [39]:
def calculation(value):
    return 2 * value

In [40]:
job_calc = pr.wrap_python_function(calculation)

In [41]:
job_calc.input.value = 100

In [42]:
job_calc.run()

The job calculation_e3cd70ecfcd6595bfd3de99721d2e84c was saved and received the ID: 159


In [43]:
job_calc.output

DataContainer({'result': 200})

# Job decorator 

In [44]:
from pyiron_base import job

In [45]:
@job
def calculation(value):
    return 2 * value

In [46]:
result = calculation(value=5)

In [47]:
result.pull()

10